# Assignment 4
## Model Comparison using Fish Dataset

### What we'll be doing
We'll compare two models, specifically **linear & polynomial** model using the arviz library.

Both models will use:

+ Width as independent variable
+ Height as dependent variable

### Let's import the required libraries

In [12]:
import pymc as pm
import pandas as pd
import arviz as az

### Reading the dataset
Let's import the fish.csv file and read 'Height' & 'Width' column.

- X: Width  
- Y: Height

In [13]:
data = pd.read_csv("fish.csv")
x = data.Width.values
y = data.Height.values

## Creating the Linear Model

Now, let's create a linear model using the variables 'Width' and 'Height'.

The linear regression model is represented as:

In [14]:
with pm.Model() as model_linear:
    α = pm.Normal('α', mu=0, sigma=1)
    β = pm.Normal('β', mu=0, sigma=10)
    ϵ = pm.HalfNormal('ϵ', 5)

    μ = α + β * x

    y_pred1 = pm.Normal('y_pred1', mu=μ, sigma=ϵ, observed=y)

    idata_linear = pm.sample(2000, idata_kwargs={'log_likelihood': True})

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [α, β, ϵ]
Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 16 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


100.00% [3000/3000 00:07<00:00 Sampling chain 0, 0 divergences]

100.00% [3000/3000 00:08<00:00 Sampling chain 1, 0 divergences]

## Creating the Polynomial Model

Now, let's create a polynomial model using the variables 'Width' and 'Height'.

The polynomial regression model is represented as:

In [15]:
with pm.Model() as model_polynomial:
    α = pm.Normal('α', mu=0, sigma=1)
    β1 = pm.Normal('β1', mu=0, sigma=10)
    β2 = pm.Normal('β2', mu=0, sigma=10)
    ϵ = pm.HalfNormal('ϵ', 5)

    μ = α + (β1 * x) + (β2 * x**2)

    y_pred2 = pm.Normal('y_pred2', mu=μ, sigma=ϵ, observed=y)

    idata_polynomial = pm.sample(2000, idata_kwargs={'log_likelihood': True})

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [α, β1, β2, ϵ]
Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 46 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


100.00% [3000/3000 00:24<00:00 Sampling chain 0, 0 divergences]

100.00% [3000/3000 00:22<00:00 Sampling chain 1, 0 divergences]

### Model comparison using ARVIZ
Now, we'll compare both our models **az.compare()** function of library arviz which will provide us with table of values including **elpd_loo, p_loo & elpd_diff**.

In [16]:
comparison_table = az.compare({ 'model_linear':idata_linear, 'model_polynomial':idata_polynomial }, method='BB-pseudo-BMA', ic="loo", scale="deviance")

In [17]:
comparison_table

,rank,elpd_loo,p_loo,elpd_diff,weight,se,dse,warning,scale
model_polynomial,0,754.369590,2.888486,0.000000,0.90147,14.099241,0.000000,False,deviance
model_linear,1,760.950569,2.522283,6.580979,0.09853,14.112204,3.787692,False,deviance


## Analysis
Let's try to answer the following questions based on the table above:

- Which model do you think is better from predictive accuracy point of view and why?
- Which model is complex and why?

## Predictive Accuracy
#### LOO-CV (Leave-One-Out Cross-Validation) and ELPD (Expected Log Pointwise Predictive Density):

Lower LOO-CV values and higher ELPD indicate better predictive accuracy.
In our case, the **model_polynomial** has a slightly higher ELPD than model_linear, indicating better predictive accuracy. This is supported by the lower LOO-CV value for model_polynomial.

#### p_LOO (Effective Number of Parameters):

A lower value suggests a more parsimonious model.
model_polynomial has a higher p_LOO, indicating more effective parameters in the model compared to model_linear.

## Model Complexity
#### LOO-CV and ELPD Difference:

The ELPD difference is the difference in predictive accuracy between the models.
A smaller ELPD difference suggests that the simpler model (with lower ELPD) is sufficient.

#### p_LOO:

A higher p_LOO suggests a more complex model.
**model_polynomial** has a higher p_LOO, indicating higher complexity.

## Conclusion
Based on our analysis

### Predictive Accuracy:

The **model_polynomial** appears to have slightly better predictive accuracy, as indicated by the slightly higher ELPD and lower LOO-CV compared to model_linear.

### Model Complexity:

The **model_polynomial** is more complex, as indicated by its higher p_LOO.